Marlon F. Souza built this code for data preprocessing of the analysis in the article "Port regionalization for agricultural commodities: Mapping exporting port hinterlands". Please give the credits if using.

The code was prepared to run in Google Colab.

## Libraries

In [ ]:
# importing basic libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# importing the library for raster files download
import requests

In [ ]:
# installing libraries 
!pip install geopandas
!pip install rasterio

In [ ]:
# importing GIS libraries
import geopandas as gpd
import fiona
import rasterio
from rasterio.merge import merge
from rasterio.plot import show
from rasterio.enums import Resampling
from rasterio.mask import mask

## Mount the Google drive and indicate the folder with the files

In [ ]:
# mounting drive
from google.colab import drive
drive.mount('/content/drive')

# defining the working directory
os.chdir('/content/drive/MyDrive/your_folder')
os.listdir('/content/drive/MyDrive/your_folder')

## Define function for downloading TOPODATA INPE files

In [ ]:
def baixar_arquivo(url, endereco, arquivo):
    resposta = requests.get(url, stream=True)
    if resposta.status_code == requests.codes.OK:
        with open(endereco, 'wb') as novo_arquivo:
                for parte in resposta.iter_content(chunk_size=256):
                    novo_arquivo.write(parte)
        print("Download of the file {} finished.".format(arquivo))
    else:
        resposta.raise_for_status()

## Define function for resampling downloaded raster files

In [ ]:
def reamostragem(tile, caminho_dest, nome_arquivo):
    resampled_tile = tile.read(
    out_shape=(tile.count, int(tile.height * upscale_factor), int(tile.width * upscale_factor)),
    resampling=Resampling.average) #downsample each tile

    transform = tile.transform * tile.transform.scale(
        (tile.width / resampled_tile.shape[-1]),
        (tile.height / resampled_tile.shape[-2])
        )

    out_meta = tile.meta.copy() # Copy the metadata
    # Update the metadata
    out_meta.update({"driver": "GTiff",
                    "height": resampled_tile.shape[1],
                    "width": resampled_tile.shape[2],
                    "transform": transform,
                    "crs": "+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs",                    
                    }
                    )
    with rasterio.open(caminho_dest+nome_arquivo+'.tif', 'w', **out_meta) as dest_f:
      dest_f.write(resampled_tile)
    print('Raster {} extraído.'.format(nome_arquivo))

## Read the file (shp or gdb) with TOPODATA tiles
### Tiles can be found in http://www.dsr.inpe.br/topodata/acesso.php
### We use Brazilian states to split the grid and avoid download issues by downloading a smaller data set each time. 

In [ ]:
# Get all the layers from the .gdb file 
layers = fiona.listlayers('TOPODATA_tiles.gdb')

layers

In [ ]:
# read brazilian states shapefile
UFs = gpd.read_file('zip:///content/drive/MyDrive/your_folder/lm_ufs_bc250_IBGE.zip!lm_ufs_Brasil.shp')
UFs.plot();

## Main code

In [ ]:
upscale_factor = 1/10 # The factor that each tile will be downsampled. For upscale use a number > 1
# For downscale use a fraction or a number between 0-1
diretorio = '/content/drive/MyDrive/your_folder/'

# loop for download, extract, and downsampling by state
for layer in layers:
    shape = UFs.loc[UFs.UF==layer].geometry
    gdf = gpd.read_file('TOPODATA_tiles.gdb',layer=layer)
    print('###################################################################')
    print('Downloading files of'+layer)

    caminho = diretorio+'TOPODATA/' # folder to save the downloaded tiles
    if not os.path.exists(caminho): # Check if the folder exists
        os.mkdir(caminho) # Create new folder if it does not exist

    caminho_dest = diretorio+'TOPODATA_downsampled_tiles_300x300/' # folder to save rasters after extraction and downsampling
    if not os.path.exists(caminho_dest): # Check if the folder exists
      os.mkdir(caminho_dest) # Create new folder if it does not exist

    caminho_final = diretorio+'mosaico_UF/' # folder to save mosaics by UFs
    if not os.path.exists(caminho_final): # Check if the folder exists
      os.mkdir(caminho_final) # Create new folder if it does not exist

    # Downloading and checking zipped raster files by UF 
    lista_id = [id[:6] for id in gdf['TOPO_ID']] # creates a list with all UF tiles limiting the ID to 6 characters
    while True: # Creates a loop that analyzes the folder where files are saved to know when the program should stop
      files_done = (os.listdir(caminho)) # List of files that were successfully downloaded
      files_UF = [s + 'ZN.zip' for s in lista_id] # List of all files covering a state
      files_remain = np.setdiff1d(files_UF, files_done) # Files that have not yet been downloaded
      print('Remaining downloads: {}'.format(len(files_remain))) # Shows the amount of files remaining
    
      if len(files_remain) == 0: # If there are no more files left, the loop will be terminated
          print('The {} download is complete! Congrats!'.format(layer))
          break # ends the loop
      
      for i in files_remain:
          url = 'http://www.dsr.inpe.br/topodata/data/geotiff/'+i # url for download
          endereco = caminho+'/'+i # path to save downloaded files
          baixar_arquivo(url, endereco, i) # download file function

    # Extracting and downsamling the raster files
    while True: # Creates a loop that analyzes the folder to know when the program should stop
      files_done = (os.listdir(caminho_dest)) # List of files that were successfully resampled
      files_UF = [s + 'ZN.tif' for s in lista_id] # List of all files of the state
      files_remain = np.setdiff1d(files_UF, files_done) # Files that have not yet been resampled
      print('Extraction and resampling remaining: {}'.format(len(files_remain))) # Shows the amount of files remaining

      if len(files_remain) == 0: # If there are no more files left, the loop will be terminated
        print('The {} downsampling is complete! Congratulations!'.format(layer))
        print('###################################################################')
        break # ends the loop for extracting and resampling
      
      for i in files_remain:    
        endereco = caminho+i[:8]+'.zip'
        raster_path = 'zip://'+endereco+'!'+i[:8]+'.tif'# extracting zip
        print(raster_path)
        tile = rasterio.open(raster_path) # opening raster files
        reamostragem(tile, caminho_dest, i[:8]) # resampling function

    # files to merge into the DEM for each UF
    raster_name = caminho_final+'DEM_300x300_'+layer+'.tif'
    if not os.path.exists(raster_name):
      src_files_to_mosaic = []
      for grade in list(set(files_UF)):
        raster_path = caminho_dest+grade[:8]+'.tif'# path of the resampled raster
        print(raster_path)
        src = rasterio.open(raster_path) # opening raster files
        src_files_to_mosaic.append(src) # gathering the files for mosaic

      mosaic, out_trans = merge(src_files_to_mosaic) # mosaic

      out_meta = src.meta.copy() # Copy the metadata
      # Update the metadata
      out_meta.update({"driver": "GTiff",
                      "height": mosaic.shape[1],
                      "width": mosaic.shape[2],
                      "transform": out_trans,
                      "crs": "+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs",                    
                      })
      
      # Write the mosaic raster to disk
      with rasterio.open(raster_name, "w", **out_meta) as dest:
        dest.write(mosaic)

    # extract by mask
    mosaico_UF_final = caminho_final+'DEM_300x300_'+layer+'_masked.tif'
    if not os.path.exists(mosaico_UF_final):
      with rasterio.open(caminho_final+'DEM_300x300_'+layer+'.tif') as mosaico:
        out_image, out_transform = mask(mosaico, shape, crop=True)
        out_meta = mosaico.meta
      
      out_meta.update({"driver": "GTiff",
                      "height": out_image.shape[1],
                      "width": out_image.shape[2],
                      "transform": out_transform})
      
      # Plot the result
      show(out_image, cmap='terrain')
      
      # Write the masked mosaic to disk
      with rasterio.open(mosaico_UF_final, "w", **out_meta) as dest:
          dest.write(out_image)

To have the final DEM raster file, the files from each state must be mosaiced and clipped with a mask of the Brazilian frontier.